In [20]:
from sklearn import linear_model
from sklearn import cross_validation
from sklearn.grid_search import GridSearchCV

def logistic_c(Xtest, ytest):
    scores = []
    k_fold = cross_validation.KFold(Xtest.shape[0],10)
    Cvalues = [10000,1000,100,10,1,0.1,0.01,0.001,0.0001]
    for c in Cvalues:
        score = 0
        for k, (train, val) in enumerate(k_fold):
            Xcross = Xtest[train]
            logit = linear_model.LogisticRegression(C=c)
            logit.fit(Xcross,np.ravel(ytest[train].toarray()))
            score += logit.score(Xtest[val],ytest.toarray()[val])            
        scores.append(score/10)
    best_value_ind = scores.index(max(scores))
        
    return Cvalues[best_value_ind]

In [3]:
from scipy.sparse import csr_matrix
from sklearn.utils import shuffle
import numpy as np
user_skills = open('user_skill','r')

col = []
row = []
data = []

counter = 0
for line in user_skills:
    line = line.strip('\n')
    skills = map(int,line.split(':')[1].split(','))
    data += ([1]*len(skills))
    row +=([counter]*len(skills))
    col += (skills)
    counter +=1

user_skills.close()
col = np.asarray(col)
row = np.asarray(row)
data = np.asarray(data)
Z = csr_matrix((data,(row, col)),shape= (counter+1,14544))

In [4]:
nrows = np.shape(Z)[0]
Z=shuffle(Z, random_state = 0)
train_size = int(0.7*nrows)
Ztr = Z[range(train_size),:]
Zts = Z[range(train_size, nrows),:]

In [5]:
import operator
skills_count = np.asarray(Z.sum(axis=0))[0]
skills_labels = []
skills_id = open('skill_id','r')
for line in skills_id:
    skills_labels.append(unicode(line.split(' :')[0],'utf-8'))
    
dictionary = dict(zip(skills_labels, skills_count.T))
x = []
y = []
for i in range(20):
    skill = max(dictionary.iteritems(), key=operator.itemgetter(1))[0]
    x.append(skill)
    y.append(dictionary.get(skill))
    del dictionary[skill]

import numpy as np
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
width = 0.75 # the width of the bars 
ind = np.arange(len(y))  # the x locations for the groups
ax.barh(ind, y, width, color="blue")
ax.set_yticks(ind+width/2)
ax.set_yticklabels(x, minor=False)
plt.title('Top 20 Skills Linkedin')
plt.xlabel('Users with the Skill')
plt.ylabel('Skill')      
plt.show()

# Modelos

In [22]:
from sklearn import linear_model
selected_skills = [29]
for skill in selected_skills:
    ytr = Ztr[:,skill]
    Xtr = csr_matrix(np.delete(Ztr.toarray(),skill,1))
    yts = Zts[:,skill]
    Xts = csr_matrix(np.delete(Zts.toarray(),skill,1))
    c = logistic_c(Xtr, ytr)
    
    #Logistic Regression model
    logit_model = linear_model.LogisticRegression(C=c)
    ytr = np.ravel(ytr.toarray())
    yts = np.ravel(yts.toarray())
    logit_model.fit(Xtr, ytr)
    
    print logit_model.score(Xtr, ytr)
    print logit_model.score(Xts, yts)
    
#TODO: implementar cross validation para escoger parámetros

0.87742169111
0.831081081081
